# Val and Marta's Final Project
## CSE360/460

### Frisbee-Catching Robot

In [ ]:
import sim
%pylab inline
user_ip = '192.168.32.1' #'127.0.0.1'

In [ ]:
class robot():
            
    def __init__(self, frame_name, motor_names=[], client_id=0):  
        # If there is an existing connection
        if client_id:
                self.client_id = client_id
        else:
            self.client_id = self.open_connection()
            
        self.motors = self._get_handlers(motor_names) 
        
        # Robot frame
        self.frame =  self._get_handler(frame_name)
            
        
    def open_connection(self):
        sim.simxFinish(-1)  # just in case, close all opened connections
        self.client_id = sim.simxStart(user_ip, 19999, True, True, 5000, 5)  # Connect to CoppeliaSim 
        
        if clientID != -1:
            print('Robot connected')
        else:
            print('Connection failed')
        return clientID
        
    def close_connection(self):    
        sim.simxGetPingTime(self.client_id)  # Before closing the connection to CoppeliaSim, make sure that the last command sent out had time to arrive.
        sim.simxFinish(self.client_id)  # Now close the connection to CoppeliaSim:
        print('Connection closed')
    
    def isConnected(self):
        c,result = sim.simxGetPingTime(self.client_id)
        # Return true if the robot is connected
        return result > 0         
        
    def _get_handler(self, name):
        err_code, handler = sim.simxGetObjectHandle(self.client_id, name, sim.simx_opmode_blocking)
        return handler
    
    def _get_handlers(self, names):
        handlers = []
        for name in names:
            handler = self._get_handler(name)
            handlers.append(handler)
        
        return handlers

    def send_motor_velocities(self, vels):
        for motor, vel in zip(self.motors, vels):
            err_code = sim.simxSetJointTargetVelocity(self.client_id, 
                                                      motor, vel, sim.simx_opmode_streaming)      
            
    def set_position(self, position, relative_object=-1):
        if relative_object != -1:
            relative_object = self._get_handler(relative_object)        
        sim.simxSetObjectPosition(clientID, self.frame, relative_object, position, sim.simx_opmode_oneshot)
        
    def simtime(self):
        return sim.simxGetLastCmdTime(self.client_id)
    
    def get_position(self, relative_object=-1):
        # Get position relative to an object, -1 for global frame
        if relative_object != -1:
            relative_object = self._get_handler(relative_object)
        res, position = sim.simxGetObjectPosition(self.client_id, self.frame, relative_object, sim.simx_opmode_blocking)        
        return array(position)
    
    def get_object_position(self, object_name):
        # Get Object position in the world frame
        err_code, object_h = sim.simxGetObjectHandle(self.client_id, object_name, sim.simx_opmode_blocking)
        res, position = sim.simxGetObjectPosition(self.client_id, object_h, -1, sim.simx_opmode_blocking)
        return array(position)
    
    def get_object_relative_position(self, object_name):        
        # Get Object position in the robot frame
        err_code, object_h = sim.simxGetObjectHandle(self.client_id, object_name, sim.simx_opmode_blocking)
        res, position = sim.simxGetObjectPosition(self.client_id, object_h, self.frame, sim.simx_opmode_blocking)
        return array(position)
    

In [ ]:
# For finding the coefficients for splines, in order to move the frisbee
def point_to_point_traj(x1, x2, v1, v2, delta_t):
    # the following is finding each coefficient for the spline
    a0 = x1
    a1 = v1
    a2 = (3*x2 - 3*x1 - 2*v1*delta_t - v2 * delta_t) / (delta_t**2)
    a3 = (2*x1 + (v1 + v2) * delta_t  - 2 * x2) / (delta_t**3)
    return a0, a1, a2, a3


In [ ]:
import sim as vrep
import time
import cv2
import numpy as np
from matplotlib.transforms import Affine2D
from pathfinding.core.diagonal_movement import DiagonalMovement
from pathfinding.core.grid import Grid
from pathfinding.finder.a_star import AStarFinder

# Tha map is 20x20 sq meters
#Lets define a grid of nxn
n = 20
#gmap = zeros(n*n) # the map is a grid of nxn

# x and y coordinates for the grid cells. Lowest and leftest point in the cell.
cell_w = 20/n
grid_x, grid_y = np.mgrid[-10:10:cell_w,-10:10:cell_w]
#print(len(grid_x))
# Convert the matrix into a vector
grid_x = grid_x.flatten()
grid_y = grid_y.flatten()

#plot(grid_x, grid_y, '+')

##get trees
tree_names = ["Tree","Tree#1","Tree#2","Tree#3","Tree#5"]
#locations = []
xzcs = []
vrep.simxFinish(-1)
#connect to the server 
clientID = vrep.simxStart(user_ip, 19999, True, True, 5000, 5)

if clientID != -1:
    print('Server is connected!')
else:
    print('Server is unreachable!')
    sys.exit(0)

for name in tree_names:
    errorcode, handle = vrep.simxGetObjectHandle(clientID,name,vrep.simx_opmode_blocking)
    errorcode, location = vrep.simxGetObjectPosition(clientID,handle,-1,vrep.simx_opmode_blocking)
    #plot(location[0],location[1],'*')
    #locations.append([round(location[0]), round(location[1])])
    loc = np.array((round(location[0]), round(location[1])))
    xc = loc[0]
    zc = loc[1]
    xzcs.append([xc,zc])
    
#print(locations)
#populate_trees(locations[0],n)
print(xzcs)

def points_cell(x, y, d):    
    X = [x, x+d, x+d, x]
    Y = [y, y, y+d, y+d]
    return X, Y

#fig = plt.figure()
#ax = fig.add_subplot(111, aspect='equal')
# plot each cell
for x, y in zip(grid_x, grid_y):
    X, Y = points_cell(x, y, cell_w)
    #cell = plt.Polygon([(xi, yi) for xi, yi in zip(X,Y)], color='0.9')
    
    #ax.add_patch(cell)

    #plot(X,Y, 'k-')
    #plot(X,Y, 'b+')    
    
#fig.canvas.draw()

l0 = (0.3/(1-0.3))  # Initial belief
gmap = l0 * ones(n*n) # Initial belief
# For each cell, check if the circle is in it.
count = 0
ret_locs = []
for i in range(n*n):
    x, y = grid_x[i], grid_y[i]
    #TODO Run this for each sphere (center and radius)
    # Corners of the cell
    X, Y = points_cell(x, y, cell_w)
    # check based on the ecuclidean distance
    for j in range(0,len(xzcs)):
        xc,zc = xzcs[j]
        dist = sqrt((xc - X)**2 + (zc - Y)**2)
        radius = 2
        k = radius / (1.5 / 2)
        # Check if At least one of the borders is within the sphere
        if((dist < radius/k).any()):
            #print(dist)
            po = 0.8  # P(mi/zt) probability of having an obstacle 
            li = log(po / (1-po)) + gmap[i] - l0
            gmap[i] = li  # P(mi/zt) 
            #print(li)
            ret_locs.append([x,y])
        else:
            po = 0.05  # P(mi/zt) probability of having an obstacle given a non-detected obstacle

            # Cells within the fov. Check if the four points are withing the FOV
            #thetas = np.arctan2(Y,X) - pi/2 - angle
            thetas = 2*pi
            if np.logical_and(-pi/6 <thetas, thetas < pi/6).all():
                li = log(po / (1-po)) + gmap[i] - l0
                gmap[i] = li

            pass
print(ret_locs)
#gmap = nan_to_num(gmap)
# normalize gmap
gmap = gmap - min(gmap)
gmap = gmap / max(gmap)

In [ ]:
import sim as vrep
import time
import cv2
import numpy as np
from random import * # for random numbers
import final_project_val as vp
vrep.simxFinish(-1)
clientID = sim.simxStart(user_ip, 19999, True, True, 5000, 5)  # Connect to CoppeliaSim
if clientID != -1:
    print('Server is connected!')
else:
    print('Server is unreachable!')
    sys.exit(0)

r = robot(motor_names = [], frame_name='Quadricopter_target')

# Move the frisbee
disc = robot('Sphere')

# Create trajectory for disc 
random_num = 0
#randint(0, 2)  # pick a number between 0 and 2

# TODO: Fix trajectories to be what we want (can be done anytime)
if random_num == 0:
    x_i, x_f, vx_i, vx_f = 0.5, 1.2, 0, 0
    y_i, y_f, vy_i, vy_f = 5.5, -4.4, 0, 0
    z_i, z_f, vz_i, vz_f = 1.5, 3, 0, 0
elif random_num == 1:
    x_i, x_f, vx_i, vx_f = 0, 0, 0, 0
    y_i, y_f, vy_i, vy_f = 0, 0, 0, 0
    z_i, z_f, vz_i, vz_f = 0, 0, 0, 0
elif random_num == 2:
    x_i, x_f, vx_i, vx_f = 0, 0, 0, 0
    y_i, y_f, vy_i, vy_f = 0, 0, 0, 0
    z_i, z_f, vz_i, vz_f = 0, 0, 0, 0
    
a0x, a1x, a2x, a3x = point_to_point_traj(x_i, x_f, vx_i, vx_f, 20)
a0y, a1y, a2y, a3y = point_to_point_traj(y_i, y_f, vy_i, vy_f, 20)
a0z, a1z, a2z, a3z = point_to_point_traj(z_i, z_f, vz_i, vz_f, 20)

# Time interval
time_steps = linspace(0, 20, 201)

res, v1 = vrep.simxGetObjectHandle(clientID, 'Vision_sensor', vrep.simx_opmode_oneshot_wait)
err, resolution, image = vrep.simxGetVisionSensorImage(clientID, v1, 0, vrep.simx_opmode_streaming)

def get_image():
    err, resolution, image = vrep.simxGetVisionSensorImage(clientID, v1, 0, vrep.simx_opmode_buffer)
    orientation = vrep.simxGetObjectOrientation(clientID, v1, -1, vrep.simx_opmode_blocking)[1][1]
    orientations.append(orientation)
    goal = r.get_position(-1)
    if err == vrep.simx_return_ok:
        #print ("image OK!!!")
        img = np.array(image,dtype=np.uint8)
        img.resize([resolution[1],resolution[0],3])
        images.append(img)
        for img in images:
            gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
            gray = cv2.medianBlur(gray,5)
            circles = cv2.HoughCircles(gray, cv2.HOUGH_GRADIENT, 1, 20, param1=30,param2=15,minRadius=0,maxRadius=0)
        j = -1 # just use the first circle
        if circles is not None:
            for circle in circles[0, :]:
                center = (circle[0], circle[1])  # In pixels
                radius = circle[2]  # In pixels
                # Plot center
                u0, v0 = 132, 132
                k = radius / (0.5 / 2)  # Pixels per meters (radius in pixels / radius in  meters)
                fov = 60  # degrees
                # number of pixels in x-axis
                npx = 264
                # maximum value in the x-axis (in meters)
                xmax = npx / (2.* k)
                # Distance to the center of the sphere (z-axis coordinate)
                zc = xmax / tan(pi/6)
                # Location of the sphere in the x-axis coordinate
                xc = ((center[0] - u0) / k)
                posx, posy, posz = r.get_position(-1)
                goal = [posx+xc,posy+zc,posz]
                #disp(posx)
                #disp(posy)
                #disp(posz)
#                     if j == -1:
#                         j = 1
#                         if xc > 0.05:

#                             r.set_position([posx + 0.2,posy, posz], -1)
#                         elif xc < -0.05:
#                             r.set_position([posx - 0.2, posy, posz], -1)
    return goal

orientations = []
images = []
circles_array = []
s = 0
a = 0
goal = r.get_position(-1)
for t in time_steps:
    tx, ty, tz = a0x + a1x * t + a2x * t**2 + a3x * t**3, a0y + a1y * t + a2y * t**2 + a3y * t**3, a0z + a1z * t + a2z * t**2 + a3z * t**3
    disc.set_position([tx, ty, tz])
    start = r.get_position(-1)
    a0, a1, a2, a3 = vp.get_pairs_for_traj(start,goal,ret_locs)
    a0thirds, a1thirds, a2thirds, a3thirds = a0[0], a1[0], a2[0], a3[0]
    #print(f"First few a0 pairs {a0[0]}, {a0[1]}, {a0[2]}")
    at = a
    x_point_traj = a0thirds[0] + a1thirds[0]*a + a2thirds[0]*(a**2) + a3thirds[0]*(a**3) #Gamma_x
    z_point_traj = a0thirds[1] + a1thirds[1]*a + a2thirds[1]*(a**2) + a3thirds[1]*(a**3) #Gamma_z
    #print(np.array([x_point_traj, y_point_traj, 0]))
    posx, posy, posz = r.get_position()
    #print(f"Position to go to: {[x_point_traj,z_point_traj, posz]}")
    if s != 0:
        r.set_position([x_point_traj,z_point_traj, posz],-1)
    a=a+1
    if s%10 == 0:
        goal = get_image()
        a=0
        
    time.sleep(.3)
    s = s + 1


In [ ]:
# scrap code (not used)

from mpl_toolkits.mplot3d import axes3d
import matplotlib.pyplot as plt

new_centers = []
circles_array = []
for img in images:
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    gray = cv2.medianBlur(gray,5)
    circles = cv2.HoughCircles(gray, cv2.HOUGH_GRADIENT, 1, 20, param1=30,param2=15,minRadius=0,maxRadius=0)
    circles_array.append(circles)
    
i = -1
for circles in circles_array:
    i = i + 1
    # for every circle we find it in our pov and plot it
    if circles is not None:
        for circle in circles[0, :]:
            center = (circle[0], circle[1])  # In pixels
            radius = circle[2]  # In pixels
            # Plot center
            u0, v0 = 132, 132
            k = radius / (0.5 / 2)  # Pixels per meters (radius in pixels / radius in  meters)
            fov = 60  # degrees

            # number of pixels in x-axis
            npx = 264

            # maximum value in the x-axis (in meters)
            xmax = npx / (2.* k)

            # Distance to the center of the sphere (z-axis coordinate)
            zc = xmax / tan(pi/6)

            # Location of the sphere in the x-axis coordinate
            xc = ((center[0] - u0) / k)
  
            plot(xc, zc, '*')
            
            # plot the transformed circles
            s = linspace(0,2*pi,100)
            cx = cos(s) + xc
            cy = sin(s) + zc
            plot(cx, cy)
            new_centers.append([xc, zc])
